In [28]:
import snowflake.connector as sf
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import joblib
from fastapi import FastAPI
from pydantic import BaseModel
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
from keras.preprocessing.sequence import TimeseriesGenerator

In [29]:
#Define the S3 bucket and Snowflake parameters
snowflake_account = 'at67872.ap-south-1.aws'
snowflake_user = 'FYP4'
snowflake_password = '!Q2w3e4r5t6y'
snowflake_database = 'COAPDATA'
snowflake_schema = 'PUBLIC'
snowflake_warehouse = 'COMPUTE_WH'
snowflake_table = 'daily_consumption'

In [30]:
conn = sf.connect(user=snowflake_user, password=snowflake_password, account=snowflake_account)
cursor = conn.cursor()
cursor.execute(f"SELECT * FROM {snowflake_database}.{snowflake_schema}.{snowflake_table} ORDER BY Date DESC")
data = cursor.fetchall()
cursor.close()
conn.close()
df = pd.DataFrame(data, columns=['Date', 'consumption'])
df.set_index('Date', inplace=True)
df.sort_index(inplace=True)

In [31]:

df

,consumption
Date,
2023-05-23,1800.0
2023-05-24,1900.0
2023-05-25,2000.0
2023-05-26,950.0
2023-05-27,1200.0
2023-05-28,1250.0
2023-05-29,1500.0
2023-05-30,1700.0
2023-05-31,1800.0


In [32]:
# Define the model parameters
n_input = 10
n_features = 1
model_file_name = 'model.h5'
#load the initial LSTM model
model = load_model(model_file_name)


In [33]:
 # Scale the data using the same scaler used for training
scaler = MinMaxScaler()
scaler.fit(df)
scaled_data = scaler.transform(df)

In [34]:
# Update the model with the new data
generator = TimeseriesGenerator(scaled_data, scaled_data, length=n_input, batch_size=1)

In [35]:
# take the last 10 and reshape
last_scaled_batch = scaled_data[-10:]
last_scaled_batch = last_scaled_batch.reshape((1, n_input, n_features))
    

In [36]:
model.fit(generator, epochs=1, verbose=0)
model.save('model.h5')

In [37]:
prediction = model.predict(last_scaled_batch)
true_prediction = scaler.inverse_transform(prediction)[0][0]

In [38]:
true_prediction

1204.4625

In [39]:
snowflake_conn = sf.connect(
        user='FYP4',
        password='!Q2w3e4r5t6y',
        account='at67872.ap-south-1.aws',
        warehouse='COMPUTE_WH',
        database='COAPDATA',
        schema='PUBLIC'
    )


In [40]:
cursor = snowflake_conn.cursor()
cursor.execute(f"INSERT INTO predict_table (date, prediction) VALUES (DATEADD(DAY, 1, (SELECT MAX(date) FROM daily_consumption)), {true_prediction})")
cursor.close()
snowflake_conn.close()